##Script Variables
These are values that could be passed into this script as configuration

In [0]:
SF_URL = "https://rg93674.us-east-2.aws.snowflakecomputing.com"
SF_DATABASE = "USER_OSUSAM28"
SF_SCHEMA = "INTERVIEW"
SF_WAREHOUSE = "INTERVIEW_WH"

AIRLINES_TABLE_NAME = "AIRLINES"
AIRPORTS_TABLE_NAME = "AIRPORTS"
FLIGHTS_TABLE_NAME = "FLIGHTS"

AIRLINE_AIRPORT_MONTHLY_FLIGHT_CNT_VW_NAME = "AIRLINE_AIRPORT_MONTHLY_FLIGHT_CNT"
ON_TIME_AIRLINE_PTG_2015_VW_NAME = "ON_TIME_AIRLINE_PTG_2015"
AIRLINE_DELAYS_VW_NAME = "AIRLINE_DELAYS"
AIRPORT_CANCEL_REASONS_VW_NAME = "AIRPORT_CANCEL_REASONS"
AIRPORT_DELAY_REASONS_VW_NAME = "AIRPORT_DELAY_REASONS"
MOST_UNIQUE_ROUTES_AIRLINE_VW_NAME = "MOST_UNIQUE_ROUTES_AIRLINE"

In [0]:
%scala

val SF_URL = "https://rg93674.us-east-2.aws.snowflakecomputing.com"
val SF_DATABASE = "USER_OSUSAM28"
val SF_SCHEMA = "INTERVIEW"
val SF_WAREHOUSE = "INTERVIEW_WH"

val CREATE_AIRLINES_TABLE_SQL = """
CREATE TABLE IF NOT EXISTS AIRLINES (IATA_CODE STRING, AIRLINE STRING)
"""

val CREATE_AIRPORTS_TABLE_SQL = """
CREATE TABLE IF NOT EXISTS AIRPORTS (IATA_CODE STRING, AIRPORT STRING, CITY STRING, STATE STRING, COUNTRY STRING, LATITUDE NUMBER, LONGITUDE NUMBER)
"""

val CREATE_FLIGHTS_TABLE_SQL = """
CREATE TABLE IF NOT EXISTS FLIGHTS (YEAR NUMBER, MONTH NUMBER, DAY NUMBER, DAY_OF_WEEK NUMBER, AIRLINE STRING, FLIGHT_NUMBER STRING, TAIL_NUMBER STRING, ORIGIN_AIRPORT STRING, DESTINATION_AIRPORT STRING, SCHEDULED_DEPARTURE STRING, DEPARTURE_TIME STRING, DEPARTURE_DELAY NUMBER, TAXI_OUT NUMBER, WHEELS_OFF STRING, SCHEDULED_TIME NUMBER, ELAPSED_TIME NUMBER, AIR_TIME NUMBER, DISTANCE NUMBER, WHEELS_ON NUMBER, TAXI_IN NUMBER, SCHEDULED_ARRIVAL NUMBER, ARRIVAL_TIME STRING, ARRIVAL_DELAY STRING, DIVERTED NUMBER, CANCELLED NUMBER, CANCELLATION_REASON STRING, AIR_SYSTEM_DELAY NUMBER, SECURITY_DELAY NUMBER, AIRLINE_DELAY NUMBER, LATE_AIRCRAFT_DELAY NUMBER, WEATHER_DELAY NUMBER)
"""

val AIRLINE_AIRPORT_MONTHLY_FLIGHT_CNT_SQL = """
CREATE OR REPLACE VIEW AIRLINE_AIRPORT_MONTHLY_FLIGHT_CNT AS (
  WITH PIVOTED AS (
    SELECT AIRLINE, AIRPORT, 
    CASE WHEN MONTH = 1 THEN 1 ELSE 0 END JAN_FLIGHTS,
    CASE WHEN MONTH = 2 THEN 1 ELSE 0 END FEB_FLIGHTS,
    CASE WHEN MONTH = 3 THEN 1 ELSE 0 END MAR_FLIGHTS,
    CASE WHEN MONTH = 4 THEN 1 ELSE 0 END APR_FLIGHTS,
    CASE WHEN MONTH = 5 THEN 1 ELSE 0 END MAY_FLIGHTS,
    CASE WHEN MONTH = 6 THEN 1 ELSE 0 END JUN_FLIGHTS,
    CASE WHEN MONTH = 7 THEN 1 ELSE 0 END JUL_FLIGHTS,
    CASE WHEN MONTH = 8 THEN 1 ELSE 0 END AUG_FLIGHTS,
    CASE WHEN MONTH = 9 THEN 1 ELSE 0 END SEPT_FLIGHTS,
    CASE WHEN MONTH = 10 THEN 1 ELSE 0 END OCT_FLIGHTS,
    CASE WHEN MONTH = 11 THEN 1 ELSE 0 END NOV_FLIGHTS,
    CASE WHEN MONTH = 12 THEN 1 ELSE 0 END DEC_FLIGHTS
    FROM (
      SELECT MONTH, AIRLINE, ORIGIN_AIRPORT AIRPORT 
      FROM "USER_OSUSAM28"."INTERVIEW"."FLIGHTS"
      UNION ALL
      SELECT MONTH, AIRLINE, DESTINATION_AIRPORT AIRPORT
      FROM "USER_OSUSAM28"."INTERVIEW"."FLIGHTS"
    )
  )
  SELECT AIRLINES.AIRLINE, 
      AIRPORTS.AIRPORT, 
      SUM(JAN_FLIGHTS) JAN_FLIGHTS,
      SUM(FEB_FLIGHTS) FEB_FLIGHTS,
      SUM(MAR_FLIGHTS) MAR_FLIGHTS,
      SUM(APR_FLIGHTS) APR_FLIGHTS,
      SUM(MAY_FLIGHTS) MAY_FLIGHTS,
      SUM(JUN_FLIGHTS) JUN_FLIGHTS,
      SUM(JUL_FLIGHTS) JUL_FLIGHTS,
      SUM(AUG_FLIGHTS) AUG_FLIGHTS,
      SUM(SEPT_FLIGHTS) SEPT_FLIGHTS,
      SUM(OCT_FLIGHTS) OCT_FLIGHTS,
      SUM(NOV_FLIGHTS) NOV_FLIGHTS,
      SUM(DEC_FLIGHTS) DEC_FLIGHTS
  FROM PIVOTED
  JOIN "USER_OSUSAM28"."INTERVIEW"."AIRLINES" AIRLINES ON AIRLINES.IATA_CODE = PIVOTED.AIRLINE
  JOIN "USER_OSUSAM28"."INTERVIEW"."AIRPORTS" AIRPORTS ON AIRPORTS.IATA_CODE = PIVOTED.AIRPORT
  GROUP BY 1,2
  ORDER BY 1 DESC, 2 DESC
)
"""

val ON_TIME_AIRLINE_PTG_2015_SQL = """
CREATE OR REPLACE VIEW ON_TIME_AIRLINE_PTG_2015 AS (
  SELECT AIRLINES.AIRLINE, CNT/TOTAL_CNT ON_TIME_PTG
  FROM (
    SELECT AIRLINE, 
      CASE WHEN ARRIVAL_DELAY > 1 THEN TRUE ELSE FALSE END IS_DELAY, 
      COUNT(*) CNT, 
      SUM(CNT) OVER (PARTITION BY AIRLINE) TOTAL_CNT
    FROM "USER_OSUSAM28"."INTERVIEW"."FLIGHTS"
    WHERE YEAR = 2015
    GROUP BY 1, 2
  ) COUNTS 
  JOIN "USER_OSUSAM28"."INTERVIEW"."AIRLINES" AIRLINES ON COUNTS.AIRLINE = AIRLINES.IATA_CODE
  WHERE IS_DELAY = FALSE
)
"""

val AIRLINE_DELAYS_SQL = """
CREATE OR REPLACE VIEW AIRLINE_DELAYS AS (
  SELECT AIRLINES.AIRLINE, CNT DELAY_CNT
  FROM (
    SELECT AIRLINE, CASE WHEN ARRIVAL_DELAY > 1 THEN TRUE ELSE FALSE END IS_DELAY, COUNT(*) CNT
    FROM "USER_OSUSAM28"."INTERVIEW"."FLIGHTS"
    GROUP BY 1, 2
  ) COUNTS
  JOIN "USER_OSUSAM28"."INTERVIEW"."AIRLINES" AIRLINES ON COUNTS.AIRLINE = AIRLINES.IATA_CODE
  WHERE IS_DELAY = TRUE
)
"""

val AIRPORT_CANCEL_REASONS_SQL = """
CREATE OR REPLACE VIEW AIRPORT_CANCEL_REASONS (AIRPORT, CANCELLATION_REASON) AS (
  WITH CANCEL_REASONS AS (
    SELECT AIRPORTS.AIRPORT, 
        CASE WHEN CANCELLATION_REASON = 'A' THEN 'Airline/Carrier' 
            WHEN CANCELLATION_REASON = 'B' THEN 'Weather' 
            WHEN CANCELLATION_REASON = 'C' THEN 'National Air System' 
            WHEN CANCELLATION_REASON = 'D' THEN 'Security' 
            ELSE 'Other' END CANCELLATION_REASON
    FROM "USER_OSUSAM28"."INTERVIEW"."FLIGHTS" FLIGHTS
    JOIN "USER_OSUSAM28"."INTERVIEW"."AIRPORTS" AIRPORTS ON FLIGHTS.ORIGIN_AIRPORT = AIRPORTS.IATA_CODE
    WHERE CANCELLATION_REASON IS NOT NULL
    GROUP BY 1,2
  )
  SELECT AIRPORT, ARRAYAGG(DISTINCT CANCELLATION_REASON) CANCELLATION_REASONS
  FROM CANCEL_REASONS
  GROUP BY 1
)
"""

val AIRPORT_DELAY_REASONS_SQL = """
CREATE OR REPLACE VIEW AIRPORT_DELAY_REASONS AS (
  WITH REASONS_UNPIVOT AS (
    SELECT ORIGIN_AIRPORT, ARRAYAGG(DISTINCT DELAY_REASON) DELAY_REASONS
    FROM (
      SELECT ORIGIN_AIRPORT, DELAY, DELAY_TYPE DELAY_REASON
      FROM "USER_OSUSAM28"."INTERVIEW"."FLIGHTS" FLIGHTS
      UNPIVOT(DELAY for DELAY_TYPE in (AIR_SYSTEM_DELAY, SECURITY_DELAY, AIRLINE_DELAY, LATE_AIRCRAFT_DELAY, WEATHER_DELAY))
    ) 
    WHERE DELAY > 0
    GROUP BY 1
  ) 
  SELECT AIRPORTS.AIRPORT, DELAY_REASONS
  FROM REASONS_UNPIVOT
  JOIN "USER_OSUSAM28"."INTERVIEW"."AIRPORTS" AIRPORTS ON REASONS_UNPIVOT.ORIGIN_AIRPORT = AIRPORTS.IATA_CODE
)
"""

val MOST_UNIQUE_ROUTES_AIRLINE_SQL = """
CREATE OR REPLACE VIEW MOST_UNIQUE_ROUTES_AIRLINE AS (
  SELECT AIRLINES.AIRLINE, COUNT(*) DESTINATION_CNT
  FROM (
    SELECT DISTINCT AIRLINE, DESTINATION_AIRPORT
    FROM "USER_OSUSAM28"."INTERVIEW"."FLIGHTS"
  ) FLIGHTS
  JOIN "USER_OSUSAM28"."INTERVIEW"."AIRLINES" AIRLINES ON FLIGHTS.AIRLINE = AIRLINES.IATA_CODE
  GROUP BY 1
)
"""

##Snowflake Connector Configuration
Python and scala objects for connecting to a snowflake instance

In [0]:
user = "osusam28"
password = "#########"

sf_options = {
  "sfUrl": SF_URL,
  "sfUser": user,
  "sfPassword": password,
  "sfDatabase": SF_DATABASE,
  "sfSchema": SF_SCHEMA,
  "sfWarehouse": SF_WAREHOUSE,
  "truncate_table": "on",
  "purge": "on"
}

In [0]:
%scala
import net.snowflake.spark.snowflake.Utils

val user = "osusam28"
val password = "#########"

val sf_options = Map(
  "sfUrl" -> SF_URL,
  "sfUser" -> user,
  "sfPassword" -> password,
  "sfDatabase" -> SF_DATABASE,
  "sfSchema" -> SF_SCHEMA,
  "sfWarehouse" -> SF_WAREHOUSE
)

##Read Uploaded Files
Files were uploaded to DBFS and are read into dataframes

In [0]:
airlines_df = spark.read.format("csv").options(header='true', inferSchema='false').load("dbfs:/FileStore/shared_uploads/osusam28@gmail.com/airlines.csv")
airports_df = spark.read.format("csv").options(header='true', inferSchema='false').load("dbfs:/FileStore/shared_uploads/osusam28@gmail.com/airports.csv")
flights_df = spark.read.format("csv").options(header='true', inferSchema='false').load("dbfs:/FileStore/shared_uploads/osusam28@gmail.com/partition_*.csv")

##Setup Snowflake Tables
Scripts to create empty tables for the source data if they don't already exist

In [0]:
%scala

Utils.runQuery(sf_options, CREATE_AIRLINES_TABLE_SQL)
Utils.runQuery(sf_options, CREATE_AIRPORTS_TABLE_SQL)
Utils.runQuery(sf_options, CREATE_FLIGHTS_TABLE_SQL)

##Write to Snowflake Tables
Use the snowflake connection to write the dataframes to the tables

In [0]:
airlines_df.write \
  .format("snowflake") \
  .options(**sf_options) \
  .option("dbtable", AIRLINES_TABLE_NAME) \
  .mode("overwrite") \
  .save()

airports_df.write \
  .format("snowflake") \
  .options(**sf_options) \
  .option("dbtable", AIRPORTS_TABLE_NAME) \
  .mode("overwrite") \
  .save()

flights_df.write \
  .format("snowflake") \
  .options(**sf_options) \
  .option("dbtable", FLIGHTS_TABLE_NAME) \
  .mode("overwrite") \
  .save()

##Create Snowflake Views
Scripts to create views off of the source tables in Snowflake

In [0]:
%scala

Utils.runQuery(sf_options, AIRLINE_AIRPORT_MONTHLY_FLIGHT_CNT_SQL)
Utils.runQuery(sf_options, ON_TIME_AIRLINE_PTG_2015_SQL)
Utils.runQuery(sf_options, AIRLINE_DELAYS_SQL)
Utils.runQuery(sf_options, AIRPORT_CANCEL_REASONS_SQL)
Utils.runQuery(sf_options, AIRPORT_DELAY_REASONS_SQL)
Utils.runQuery(sf_options, MOST_UNIQUE_ROUTES_AIRLINE_SQL)

##Read In Data From Views
Read data from views into dataframes

In [0]:
flight_cnt_df = spark.read \
  .format("snowflake") \
  .options(**sf_options) \
  .option("dbtable",  AIRLINE_AIRPORT_MONTHLY_FLIGHT_CNT_VW_NAME) \
  .load()

on_time_df = spark.read \
  .format("snowflake") \
  .options(**sf_options) \
  .option("dbtable",  ON_TIME_AIRLINE_PTG_2015_VW_NAME) \
  .load()

delays_df = spark.read \
  .format("snowflake") \
  .options(**sf_options) \
  .option("dbtable",  AIRLINE_DELAYS_VW_NAME) \
  .load()

cancel_rns_df = spark.read \
  .format("snowflake") \
  .options(**sf_options) \
  .option("dbtable",  AIRPORT_CANCEL_REASONS_VW_NAME) \
  .load()

delay_rns_df = spark.read \
  .format("snowflake") \
  .options(**sf_options) \
  .option("dbtable",  AIRPORT_DELAY_REASONS_VW_NAME) \
  .load()

unique_routes_df = spark.read \
  .format("snowflake") \
  .options(**sf_options) \
  .option("dbtable",  MOST_UNIQUE_ROUTES_AIRLINE_VW_NAME) \
  .load()

##Show the Data!
Visualize each view's dataframe

####Total number of flights by airline and airport on a monthly basis

In [0]:
display(flight_cnt_df)

####On time percentage of each airline for the year 2015 (on time = delay <= 1 min)

In [0]:
on_time_df_pandas = on_time_df.toPandas()
on_time_df_pandas['ON_TIME_PTG'] = on_time_df_pandas['ON_TIME_PTG'].astype(float)
ax = on_time_df_pandas.plot.bar(x='AIRLINE', y='ON_TIME_PTG', rot=90)

####Airlines with the largest number of delays (delay = delay > 1 min)

In [0]:
delays_df_pandas = delays_df.toPandas()
delays_df_pandas['DELAY_CNT'] = delays_df_pandas['DELAY_CNT'].astype(int)
ax = delays_df_pandas.plot.bar(x='AIRLINE', y='DELAY_CNT', rot=90)

####Cancellation reasons by airport

In [0]:
display(cancel_rns_df.orderBy('AIRPORT'))

####Delay reasons by airport

In [0]:
display(delay_rns_df.orderBy('AIRPORT'))

####Airline with the most unique routes

In [0]:
display(unique_routes_df)